In [ ]:
import seaborn as sns
import pandas as pd

# Basic Comparison of Top Most Popular Subreddits Month-to-Month

In order to build the community2vec models, we kept the top 10K subreddits in each month by number of comments posted. Since the top subreddits will change over time, we need to know quickly and by how much. This can be determined using the [Jaccard Index](https://en.wikipedia.org/wiki/Jaccard_index) of the top subreddits between each month. Let $S_1$ and $S_2$ be the top 10K subreddits for months 1 and 2 respectively, then the similarity and diversity of top subreddits between those months is $\frac{|S_1 \cap S_2|}{|S_1 \cup S_2|}$.

In [ ]:
# TODO Jaccard index for subreddits used to train community2vec models for each month in our year's worth of Reddit data

# Month-to-month Cluster Stability Comparison
We'd like to measure how much variation there is between the k-means clusters of subreddits based on the community2vec embeddings created from each month of Reddit comment data. 

Here, we use a single community2vec model for each month (don't vary the community2vec models). Each month's community2vec model is used to train 10 k-means models where the random seed for the cluster model is varied, then the inter- and intra-month stability for each cluster model is measured. We also vary the number of clusters allowed for each. For each number of clusters, 

The intra-month values (training multiple k-means models from same community2vec model for a single month) will tell us how sensitive the k-means clusters are to random initialization within the same time range and community2vec model. We expect these values to be low relative to inter-month values. If they are high, it's an indication that the clusterings are sensitive to random initializations and indicates they may be difficult to use for research purposes. 

The inter-month values (training multiple k-means models for two separate months and doing many pairwise comparisons) will tell us how much the clusterings change over time. In most cases, we expect these values to be low when comparing two months that are close together in time, but higher for months that are far apart. In other words, these metrics should give an indication of how much Reddit clusterings change as time goes on. However, there's the possibility for these values to be high when drastic changes occur on Reddit, such as when certain subreddits spike in popularity or are banned. In such cases, the inter-month stability metrics could serve as an indicator of large changes in Reddit's "geography". 